![alt text for screen readers](https://intro-to-btt-using-python-assets.s3.amazonaws.com/bladesight_logo_horizontal_ORIGINAL.jpg).
# Chapter 9: Circumferential Fourier Fit (CFF) Method

In [1]:
# Run this cell if you have not installed the `bladesight` package yet
%pip install bladesight
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# If plotly is not installed
%pip install plotly
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

# Imports


In [2]:
from bladesight import Datasets
from bladesight.btt import get_rotor_blade_AoAs, get_blade_tip_deflections_from_AoAs
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from typing import List, Tuple, Dict, Union

## Getting the dataset

In [3]:
ds = Datasets["data/intro_to_btt/intro_to_btt_ch05"]
df_opr_zero_crossings = ds['table/opr_zero_crossings']
df_prox_1 = ds['table/prox_1_toas']
df_prox_2 = ds['table/prox_2_toas']
df_prox_3 = ds['table/prox_3_toas']
df_prox_4 = ds['table/prox_4_toas']

BLADE_COUNT = 5
RADIUS = 164000

rotor_blade_AoA_dfs = get_rotor_blade_AoAs(
    df_opr_zero_crossings,
    [df_prox_1, df_prox_2, df_prox_3, df_prox_4],
    np.cumsum(np.deg2rad(np.array([19.34, 19.34, 19.34]))),
    BLADE_COUNT
)
tip_deflection_dfs = []
for df_AoAs in rotor_blade_AoA_dfs:
    df_tip_deflections = get_blade_tip_deflections_from_AoAs(
        df_AoAs,
        RADIUS,
        11,
        2,
        0.5
    )
    tip_deflection_dfs.append(df_tip_deflections)
df_resonance_window = tip_deflection_dfs[0].query("n >= 500 and n <= 600")
EO = 8


If you use this dataset in published work, please use the below citation:

Diamond, D.H. (2023) Introduction to Blade Tip Timing using Python. Available at: docs.bladesight.com (Accessed: 14 October 2023). 

Link to paper: docs.bladesight.com


## Single Revolution Case


In [ ]:
def cff_method_single_revolution(
    df_blade : pd.DataFrame,
    theta_sensor_set : List[float],
    EO : int,
    signal_suffix : str = "_filt" 
) -> pd.DataFrame: 
    PROBE_COUNT = len(theta_sensor_set)
    tip_deflection_signals = [
        f"x_p{i_probe + 1}{signal_suffix}" 
        for i_probe in range(PROBE_COUNT)
    ]
    theta_sensors = np.array(theta_sensor_set)
    A = np.ones((PROBE_COUNT, 3))
    A[:, 0] = np.sin(theta_sensors * EO)
    A[:, 1] = np.cos(theta_sensors * EO)

    A_pinv = np.linalg.pinv(A) 
    B = A_pinv.dot(
        df_blade.loc[:, tip_deflection_signals].values.T
    ) 
    df_cff = pd.DataFrame(B.T, columns=["A", "B", "C"]) 
    df_cff["X"] = np.sqrt(df_cff["A"]**2 + df_cff["B"]**2)
    df_cff["phi"] = np.arctan2(df_cff["A"], df_cff["B"])
    df_cff["n"] = df_blade["n"].values
    df_predicted_targets = pd.DataFrame(
        A.dot(B).T, 
        columns=[
            col + "_pred" 
            for col 
            in tip_deflection_signals
        ]
    ) 
    df_cff = pd.concat([df_cff, df_predicted_targets], axis=1)
    return df_cff


In [ ]:
#%%timeit
# Uncomment the above line to time the function 👆 
PROBE_COUNT = 4
df_cff_params = cff_method_single_revolution(
    df_resonance_window,
    [
        df_resonance_window[f"AoA_p{i_probe + 1}"].median()
        for i_probe in range(PROBE_COUNT)
    ],
    EO
)


In [ ]:
PROBE_COUNT = 4
df_cff_params = cff_method_single_revolution(
    df_resonance_window,
    [
        df_resonance_window[f"AoA_p{i_probe + 1}"].median()
        for i_probe in range(PROBE_COUNT)
    ],
    EO
)
for i_probe in range(PROBE_COUNT):
    predicted_tip_deflections = df_cff_params[f"x_p{i_probe+1}_filt_pred"].values
    
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=df_resonance_window['n'],
            y=df_resonance_window[f"x_p{i_probe+1}_filt"].values,
            name='Measured tip deflections, probe ' + str(i_probe + 1),
            mode='markers+lines'
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df_cff_params['n'],
            y=predicted_tip_deflections,
            name='CFF predicted tip deflections, probe ' + str(i_probe + 1),
            mode='markers+lines'
        )
    )

    fig.update_layout(
        title=f"Tip deflections, probe {i_probe + 1}",
        xaxis_title="Revolution number",
        yaxis_title="Tip deflection [μm]"
    )
    
    fig.show()

## Compare against SDoF fit

In [ ]:
from bladesight.btt.infer import perform_SDoF_fit
from bladesight.btt.infer.sdof import get_X, get_phi

In [ ]:
SDoF_params = perform_SDoF_fit(
    df_resonance_window, 
    500,
    600,
    [8]
)

In [ ]:
EO = 8
sdof_X = get_X(
    df_resonance_window['Omega'].values * EO,
    SDoF_params['omega_n'] * 2 * np.pi,
    SDoF_params['zeta'],
    SDoF_params['delta_st']
)
sdof_phi = get_phi(
    df_resonance_window['Omega'].values * EO,
    SDoF_params['omega_n'] * 2 * np.pi,
    SDoF_params['zeta'],
) - SDoF_params["phi_0"]

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_resonance_window['n'],
        y=sdof_X,
        mode="markers+lines",
        name="SDoF X"
    )
)

fig.add_trace(
    go.Scatter(
        x=df_resonance_window['n'],
        y=df_cff_params["X"],
        mode="markers+lines",
        name="CFF X"
    )
)
fig.add_trace(
    go.Scatter(
        x=df_resonance_window['n'],
        y=df_cff_params["C"],
        mode="markers+lines",
        name="CFF C"
    )
)

fig.update_layout(
    title="SDoF X vs CFF X",
    xaxis_title="Revolution number",
    yaxis_title="Vibration amplitude [μm]"
)


fig.show()


In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_resonance_window['n'],
        y=sdof_phi % (2 * np.pi),
        mode="markers+lines",
        name="SDoF phi"
    )
)

fig.add_trace(
    go.Scatter(
        x=df_resonance_window['n'],
        y=df_cff_params["phi"] % (2 * np.pi),
        mode="markers+lines",
        name="phi"
    )
)

fig.update_layout(
    title="SDoF phi vs CFF phi",
    xaxis_title="Revolution number",
    yaxis_title="Phase [rad]"
)

fig.show()


In [ ]:
omega_n_568 = df_resonance_window.query("n == 568")["Omega"].iloc[0]*EO
omega_n_567 = df_resonance_window.query("n == 567")["Omega"].iloc[0]*EO
print("CFF omega_n @ n=568: {:.3f} Hz".format(omega_n_568 / (2*np.pi)))
print("CFF omega_n @ n=567: {:.3f} Hz".format(omega_n_567 / (2*np.pi)))
print("SDoF omega_n       : {:.3f} Hz".format(SDoF_params["omega_n"]))

## Estimating the EO

In [ ]:
PROBE_COUNT = 4
EOs = np.arange(1, 17)
errors = []
for EO in EOs:
    df_cff_params = cff_method_single_revolution(
        df_resonance_window,
        [
            df_resonance_window[f"AoA_p{i_probe + 1}"].median()
            for i_probe in range(PROBE_COUNT)
        ],
        EO
    )
    error = 0
    for i_probe in range(PROBE_COUNT):
        error += np.sum(
            (
                df_cff_params[f"x_p{i_probe+1}_filt_pred"].values 
                - df_resonance_window[f"x_p{i_probe+1}_filt"].values
            )**2
        )
    errors.append(error)
print("Most likely EO:", EOs[np.argmin(errors)])


In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=EOs,
        y=errors
    )
)
fig.update_layout(
    title="Error vs EO",
    xaxis_title="EO",
    yaxis_title="Error"
)
fig.show()


## Coding exercises

### 1. Multiple Revolution Case

In [ ]:
def cff_method_multiple_revolutions(
    df_blade : pd.DataFrame,
    theta_sensor_set : List[float],
    EO : int,
    extra_revolutions : int,
    signal_suffix : str = "_filt" 
) -> pd.DataFrame:
    ...
    # Please complete me!

### 2. Writing a function we can use

In [ ]:
def perform_CFF_fit(
    df_blade : pd.DataFrame,
    n_start : int,
    n_end : int,
    EOs : List[int] = np.arange(1, 20),
    extra_revolutions : int = 1
) -> Dict[str, Union[pd.DataFrame, int]]:
    ...
    # Please complete me!